# Setup

In [7]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.4 is required in this notebook
# Earlier 2.x versions will mostly work the same, but with a few bugs
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.4"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [10]:
from tensorflow import keras
# K = keras.backend
# K.square(K.transpose(t)) + 10

## 12.3.9 カスタム訓練ループ

本題に入る前に、407頁　12.5 演習問題8　とその回答を見ると以下の通り

◆演習問題8:<br>独自のカスタム訓練ループを書かなければならないユースケースを説明しなさい。<br>
◆答え:<br>
カスタム訓練ループの独自実装はかなり高度な作業なので、どうしても必要なとき以外は避けるべきだ。Kerasは、カスタム訓練ループを書かなくても訓練をカスタマイズできるツールとして、コールバック、カスタム正則化器、カスタム制約、カスタム損失関数などを提供している。これらでカスタマイズできるときにはこれらを使い、カスタム訓練ループを作ろうなどとは考えない方がよい。カスタム訓練ループの方が、実装ミスを起こしやすく、書いたコードを再利用しにくい。
しかし、ワイド・アンド・ディープ論文
(
https://homl.info/widedeep
)
のように、ニューラルネットワークの異なる部分で異なるオプティマイザを使いたい場合などは、どうしてもカスタム訓練ループを書くことが必要になる。訓練の仕組みを正確に理解したいときやデバッグを目的とするときもカスタム訓練ループが役に立つ。

In [61]:
keras.backend.clear_session()  #メモリ解放
np.random.seed(42)
tf.random.set_seed(42)

まず、単純なモデルを作ろう。手作業で訓練ループを処理するので、モデルをコンパイルする必要はない。

In [62]:
l2_reg = keras.regularizers.l2(0.05) #正則化 11.4.1
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

正則化<br>
https://zero2one.jp/learningblog/yobinori-collab-regularization/ <br>
https://toeming.hatenablog.com/entry/2020/04/03/000925

次に、訓練セットからインスタンスのバッチを無作為に抽出してくる簡単な関数を作る（13 章で
取り上げるデータAPIを使えばもっとよい方法が得られる）。

In [131]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

random.randint
https://www.sejuku.net/blog/67872

In [150]:
np.random.randint(3,8,size = (4,8))

array([[4, 5, 5, 7, 4, 6, 5, 6],
       [6, 6, 6, 4, 6, 5, 5, 3],
       [4, 6, 7, 3, 6, 7, 3, 5],
       [5, 3, 7, 5, 7, 3, 3, 6]])

In [149]:
np.random.randint(3,size = (4,8))

array([[0, 0, 1, 1, 2, 1, 1, 2],
       [1, 2, 1, 2, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1, 0, 0, 1]])

ステップ数、ステップの総数、エポック開始からの平均損失（つまり、これの計算のためにMean
指標を使う）その他の指標で訓練ステータスを表示する関数も定義しよう。

In [190]:
def print_status_bar_temp(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
#     end = "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics,end=end)

In [205]:
import time

mean_loss = keras.metrics.Mean(name="loss") # https://stackoverflow.com/questions/62300579/what-does-the-name-of-a-keras-metric-do
# mean_loss = keras.metrics.Mean()
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar_temp(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 - loss: 0.0900 - mean_square: 858.5000


A fancier version with a progress bar:

In [212]:
def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
#     print(fmt)
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

In [213]:
progress_bar(3500, 10000, size=10)

' 3500/10000 [===>......]'

In [218]:
def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
#     end = "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)

In [221]:
mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)

50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000


Pythonの文字列整形の書式に慣れていないことでもない限り、このコードは自明だろう。{:.4f}
は、小数点以下4 桁で浮動小数点数を表示する。そして\r（キャリッジリターン）とend=""を
併用すると、同じ行に必ずステータスバーが表示されるようになる。Jupyter ノートブックでは、
print_status_bar() 関数にはプログレスバーが含まれるが、代わりに手軽なtqdm ライブラリ
を使ってもよい。
上記の準備のもと、仕事に取り掛かろう。まず、ハイパーパラメータを定義し、オプティマイザ、
損失関数、指標（この場合はただのMAE）を選ぶ。

In [31]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [222]:
# データセットの取り込み
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [227]:
X_batch, y_batch = random_batch(X_train_scaled, y_train)

print(f'X_train_scaled_sahpe : {X_train_scaled.shape}')
print(f'y_train_sahpe : {y_train.shape}\n')
print(f'X_batch : {X_batch.shape}')
print(f'y_batch : {y_batch.shape}\n')

X_train_scaled_sahpe : (11610, 8)
y_train_sahpe : (11610, 1)

X_batch : (32, 8)
y_batch : (32, 1)



In [34]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]

In [228]:
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses) #正則化? https://stackoverflow.com/questions/56693863/why-does-model-losses-return-regularization-losses
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

Epoch 1/5
11610/11610 [==============================] - loss: 0.6949 - mean_absolute_error: 0.5543
Epoch 2/5
11610/11610 [==============================] - loss: 0.6481 - mean_absolute_error: 0.5228
Epoch 3/5
11610/11610 [==============================] - loss: 0.6256 - mean_absolute_error: 0.5115
Epoch 4/5
11610/11610 [==============================] - loss: 0.6172 - mean_absolute_error: 0.5070
Epoch 5/5
11610/11610 [==============================] - loss: 0.6446 - mean_absolute_error: 0.5138


● 2つのネストされたループを作っている。1つはエポック、もう1つはエポック内のバッチの
ためのものである。<br>
● 次に、訓練セットから無作為にバッチを抽出している。<br>
● tf.GradientTape() ブロックでは、1 つのバッチの予測をし（モデルを関数として使っ
て）。損失を計算している。損失は、メインの損失にほかの損失（このモデルでは、レイヤご
とに1 つの正則化ロスがある）を加えたものである。mean_squared_error() 関数は、イ
ンスタンスあたり1つの損失を返すので、tf.reduce_mean()を使ってバッチ全体の平均を計
算する（個々のインスタンスに異なる重みを適用したい場合には、ここでそうする）。正則
化ロスはすでに単一のスカラにまとめてあるので、両方を加算すればよい（形とデータ型が
同じ複数のテンソルの総和を計算するtf.add_n()を使って）。<br>
● 次に、個々の訓練変数についての（すべての変数ではなく）損失の勾配をテープに計算して
もらい、勾配降下ステップを実行するためにオプティマイザに適用する。<br>
● そして、平均損失と指標（現在のエポック全体の）を更新し、ステータスバーを表示する。<br>
● 各エポックの終了時に再度ステータスバーを表示して処理が終わったように見せ†13、改行
文字を表示する。そして平均損失と指標をリセットする。

オプティマイザのハイパーパラメータのclipnorm かclipvalue を設定すれば、これは自動的に行ってくれる。勾配にさらに変換を加えたい場合には、apply_gradients()メソッドを呼び出
す前にすればよい。
モデルに重みの制約を加えた場合（レイヤを作るときにkernel_constraint かbias_con
straint を設定して）、apply_gradients() を実行した直後に制約を適用するように訓練ループ
を書き換える必要がある。<br><br>


In [41]:
'''
for variable in model.variables:
    if variable.constraint is not None:
        variable.assign(variable.constraint(variable))
'''

'\nfor variable in model.variables:\n    if variable.constraint is not None:\n        variable.assign(variable.constraint(variable))\n'

もっとも重要なのは、この訓練ループが訓練中とテスト中とで異なる動作をするレイヤ（たと
えば、BatchNormalizationやDropout）を処理していないことである。これを処理するために
は、training=True を指定してモデルを呼び出し、この指定を必要とするすべてのレイヤに指定
を伝えるようにする必要がある。
ご覧のように、正しく処理するためにしなければならないことはたくさんあり、簡単にミスが起
きる。しかし、その反面、自分ですべてを制御できるわけであり、どちらを取るかはあなた次第だ。
モデル†14と訓練アルゴリズムのあらゆる部分のカスタマイズの方法がわかったので、
TensorFlow の自動グラフ生成機能の使い方を見ておこう。これを使えばカスタムコードのス
ピードを大きく上げることができる。TensorFlow がサポートするあらゆるプラットフォームへの
可搬性も得られる。

In [229]:
try:
    from tqdm.notebook import trange
    from collections import OrderedDict
    with trange(1, n_epochs + 1, desc="All epochs") as epochs:
        for epoch in epochs:
            with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
                for step in steps:
                    X_batch, y_batch = random_batch(X_train_scaled, y_train)
                    with tf.GradientTape() as tape:
                        y_pred = model(X_batch)
                        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                        loss = tf.add_n([main_loss] + model.losses)
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    for variable in model.variables:
                        if variable.constraint is not None:
                            variable.assign(variable.constraint(variable))                    
                    status = OrderedDict()
                    mean_loss(loss)
                    status["loss"] = mean_loss.result().numpy()
                    for metric in metrics:
                        metric(y_batch, y_pred)
                        status[metric.name] = metric.result().numpy()
                    steps.set_postfix(status)
            for metric in [mean_loss] + metrics:
                metric.reset_states()
except ImportError as ex:
    print("To run this cell, please install tqdm, ipywidgets and restart Jupyter")

All epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/362 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/362 [00:00<?, ?it/s]

## 12.4 TensorFlow関数とグラフ

TensorFlow 1では、グラフは避けられないものだったが（そのため、グラフの複雑さも）、それはグラフがTensorFlowのAPIの中心的な部分だったからである。TensorFlow 2でもグラフは残っているが、中心ではなくなり、ずっと簡単に使えるようになった。いかに簡単かを示すために、入力の3 乗を計算する簡単な関数で試してみよう。

In [230]:
def cube(x):
    return x ** 3

この関数は、整数や浮動小数点数などのPython の値を渡して呼び出せる。そして、テンソルを
渡して呼び出すこともできる。

In [231]:
cube(2)

8

In [232]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

では、tf.function()を使ってこのPython 関数をTensrFlow 関数に変換しよう。

In [50]:
tf_cube = tf.function(cube)
tf_cube

このTF 関数は、元のPython 関数とまったく同じように使え、まったく同じ結果を返す（ただ
し、テンソルという形で）。

In [51]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [52]:
tf_cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

舞台裏では、tf.function()はcube()関数が行う計算を解析し、同等の計算グラフを生成して
いる。ご覧のように、面倒なことではない（仕組みについてはすぐあとで説明する）。tf.function
はデコレータとしても使える。実際、このような使い方の方が多い。

In [54]:
@tf.function
def tf_cube(x):
    return x ** 3

元のPython 関数は、必要ならTF関数のpython_function属性からアクセスできる。

In [55]:
tf_cube.python_function(2)

8

TensorFlow は、使われていないノードを切り落とし、式を単純化し（たとえば、1 + 2は3 に置
き換えられる）、その他さまざまなことをして計算グラフを最適化する。そして、最適化された計
算グラフが完成すると、TF関数は適切な順序で（可能であれば並列に）グラフ内のオペレーショ
ンを効率よく実行する。そのため、複雑な計算をするときは（特にTF関数は）もとのPython関数
よりもずっと高速に実行される†15。ほとんどの場合、それ以上のことを知る必要はない。Python
関数を高速化したければ、TF関数に変換せよというだけである。
しかも、独自の損失関数、指標、レイヤ、その他の独自関数を書き、それをKeras モデルで使っ
たときには（この章全体でしてきたように）、Keras が自動的にその関数をTF関数に変換してくれ
るので、tf.function()を使う必要はない。

カスタムレイヤやカスタムモデルを作るときにdynamic=True を指定すると、Keras は
Python 関数をTF 関数に変換しなくなる。モデルのcompile() メソッドを呼び出すとき
に、run_eagerly=Trueを指定しても同じ効果が得られる。

デフォルトでは、TF 関数は、入力の形とデータ型の組み合わせごとに新しいグラフを生成
し、その後の呼び出しのためにキャッシングする。たとえば、tf_cube(tf.constant(10))
を呼び出すと、[] という形のint32 テンソルのための新しいグラフが生成される。その
あとでtf_cube(tf.constant(20)) を呼び出した場合には同じグラフが再利用されるが、
tf_cube(tf.constant([10, 20])) を呼び出すと、形が[2] のint32 テンソルのために新し
いグラフが生成される。これがTF関数のポリモーフィズム（さまざまな引数型と形）への対処方
法である。しかし、これが当てはまるのはテンソル引数だけだ。TF 関数にPython の数値を渡す
と、値ごとに新しいグラフが生成される（たとえば、tf_cube(10)とtf_cube(20)のふたの呼び
出しは2 つのグラフを生成する）。

Pythonの異なる数値を指定して何度もTF関数を呼び出すと、多くのグラフが生成され、プロ
グラムのスピードは落ち、膨大な量のRAMを消費する（メモリを開放するためには、TF関数
を削除しなければならない）。そこで、Pythonの数値を引数として使うのは、レイヤのニュー
ロン数といったハイパーパラメータのように、ごく少数の値しか使われないときだけにすべき
だ。そうすれば、TensorFlow は少しずつ異なるモデルをうまく最適化できるようになる。

## 12.4.1 自動グラフとトレーシング

では、TensorFlow はどのようにしてグラフを生成しているのだろうか。まず、Python 関数の
ソースコードを解析して、forループ、whileループ、if文のほか、break、continue、return
文も含めたフロー制御構文を捕捉する。この最初のステップは自動グラフ（AutoGraph）と呼ば
れている。TensorFlow がソースコードを解析しなければならないのは、Python が制御フロー構
文を捕捉するための方法をほかに提供していないからである。Python には、+ や*といった演算
子を捕捉するための__add__()、__mul__()といったマジックメソッドはあるが、__while__()
とか__if__() といったマジックメソッドはない。自動グラフは、関数のコードを解析すると、そ
れらフロー制御文を適切なTensorFlow オペレーション（ループのためのtf.while_loop()、if
文のためのtf.cond() など）に置き換えた関数のアップグレードバージョンを出力する。たと
えば、図12-4 では、自動グラフはPython 関数のsum_squares() 関数のソースコードを解析し、
tf__sum_squares() 関数を出力している。この関数では、for ループはloop_body() 関数（内
容は、もとのforループの本体になっている）の定義とそのあとのfor_stmt()関数呼び出しに置
き換えられている。この呼び出しは、計算グラフのなかで適切なtf.while_loop()オペレーショ
ンになる。
次に、TensorFlowはこの「アップグレード」された関数を呼び出すが、このときに渡すのは引数
ではなく、シンボリックテンソル（symbolic tensor）、すなわち値のない名前、形、データ型だけ
のテンソルである。たとえば、sum_squares(tf.constant(10)) を呼び出すと、形が[] でデー
タ型がint32 のシンボリックテンソルを引数としてtf__sum_squares() 関数が呼び出される。そ
して、関数はグラフモード（graph mode）で実行される。つまり、個々のTensorFlow オペレーションは、自分自身と出力テンソルを表すノードをグラフに追加していく（イーガー実行：eager
execution とか、イーガーモード：eager modeと呼ばれる通常実行とは異なる）。グラフモードで
は、TensorFlow オペレーションは実際の計算をしない。グラフモードがデフォルトモードだった
TensorFlow 1 を知っていれば、これはおなじみの形だろう。図12-4 では、tf__sum_squares()
関数はシンボリックテンソル（この場合は、形が[] のint32 テンソル）を引数として呼び出されて
おり、トレーシングの過程で最終的なグラフが生成されている。ノードはオペレーションを表し、
矢印はテンソルを表す（生成される関数、グラフはともに単純化されている）

![12_4](12_4.png)

tf.autograph.to_code(sum_squares.python_function) を呼び出せば、生成さ
れた関数のソースコードを表示できる。きれいな表示を意図したものではないが、デバッグの
役に立つことはあるだろう。

## 12.4.2 TF関数のルール

ほとんどの場合、TensorFlow オペレーションを実行するPython 関数をTF関数に変換するのは
簡単である。@tf.functionでデコレートするか、Keras に任せるかでよい。しかし、尊重すべき
ルールがいくつかある。

- 外部ライブラリ（NumPyや標準ライブラリさえこれに含まれる）を呼び出す場合、それはトレーシング時に実行されるだけである。外部ライブラリはグラフの一部にはならない。実際、TensorFlowグラフには、TensorFlowの構成要素（テンソル、オペレーション、変数、データセットなど）しか入れられない。そのため、np.sum()ではなくtf.reduce_sum()、組み込みのsorted() 関数ではなくtf.sort()を使うようにしなければならない（本当にトレーシング時だけコードを実行したい場合を除き）。これはさらに次のような意味を持っている。
    - np.random.rand() を返すだけのTF 関数f(x) を定義すると、乱数は関数がトレーシングされるときだけ生成されるため、f(tf.constant(2.)) とf(tf.constant(3.)) は同じ乱数を返すが、f(tf.constant([2., 3.])) は別の乱数を返す。np.random.rand() をtf.random.uniform([]) にすれば、オペレーションはグラフの一部になるため、乱数は呼び出しのたびに生成されるようになる。
    - 非TensorFlowコードに副作用（何かのログを書くとか、Pythonカウンタを更新するとか）がある場合、その副作用は関数がトレーシングされるときに発生するだけであり、TF関数を呼び出すたびに発生するとは考えてはならない。
    - tf.py_function()オペレーションは任意のPythonコードをラップできるが、そんなことをするとTensorFlow がそのコードに対してグラフを最適化できなくなるため、処理性能が下がる。また、そのグラフはPython が使える（そして適切なライブラリがインストールしている）プラットフォームでなければ実行できなくなるため、可搬性が下がる。
- ほかのPython 関数やTF関数を呼び出すことはできるが、TensorFlow がそのオペレーションを計算グラフ化するので、それらは同じルールに従っていなければならない。なお、それらの関数は、@tf.functionでデコレートする必要はない。
- 関数がTensorFlow変数（または、データセットやキューなどのステートフルなTensorFlowオブジェクト）を作る場合、それは最初の呼び出しだけで作らなければならない。そうでなければ例外が起きる。変数はTF関数外で作ることが望ましい（たとえば、カスタムレイヤのbuild() メソッド内）。変数に新しい値を代入したい場合、=演算子ではなく、変数のassign()メソッドを呼び出すようにしなければならない。
- Python関数のソースコードは、TensorFlowからアクセスできるようにしておかなければならない。ソースコードにアクセスできなければ（たとえば、ソースコードへのアクセスを与えないPython シェル内で関数を定義した場合や、本番環境にコンパイル済みの*.pyc ファイルしかデプロイしていない場合）、グラフ生成処理は失敗するか、機能を限定される。
- TensorFlow は、テンソルかデータセットを反復処理するfor ループしか捕捉しない。そのため、for i in range(__x__) ではなく、for i in tf.range(__x__) を使うようにしなければ、ループはグラフ内に取り込まれなくなり、トレーシング時に実行されることになる（たとえば、ニューラルネットワークの各層を作るときなどのように、グラフの構築のためのforループならこれが適切な動作かもしれない）
- いつもと同じように、性能上の理由から、可能な限りループを使わずベクトル化実装を使うようにする。

では、まとめに入ろう。この章では、TensorFlowの簡単な概要紹介からスタートし、テンソル、
オペレーション、変数、特殊データ構造などのTensorFlow の低水準API を見てから、これらの
ツールを使ってtf.kerasのほぼすべてのコンポーネントをカスタマイズした。最後に、TF関数が性
能を引き上げる仕組み、自動グラフとトレーシングでグラフを作る過程、TF関数を書くときに従
うべきルールを説明した（生成されたグラフを探ってみるなど、ブラックボックスをもう少し開け
たい場合には、付録G で説明されている技術的な細部を参照のこと）。
次章では、TensorFlow で効率よくデータをロード、前処理する方法を見ていく。

## G.1 TF関数と具象関数

TF 関数はポリモーフィックで、データ型や形が異なる入力をサポートする。たとえば、次の
tf_cube()関数について考えてみよう。

In [130]:
@tf.function
def tf_cube(x):
    return x ** 3

新しいデータ型と形の組み合わせでTF 関数を呼び出すたびに、TF 関数はこの組み合わ
せのための独自グラフを持つ新しい具象関数（concrete function）を生成する。すでに扱っ
たことのある入力シグネチャでTF 関数が呼び出されたときには、以前生成された具象関
数が使われる。たとえば、tf_cube(tf.constant(3.0)) を呼び出すと、TF 関数はtf_cube
(tf.constant(2.0)) のために使ったのと同じ具象関数（32 ビット浮動小数点数のスカラテン
ソル用）を再利用する。しかし、tf_cube(tf.constant([2.0])) やtf_cubetf.constant
(([3.0])) を呼び出すと新しい具象関数（形が[1] の32 ビット浮動小数点数用テンソル）
を作り、tf_cube(tf.constant([[1.0, 2.0], [3.0, 4.0]])) を呼び出すとさらに新し
い具象関数（ 形が[2,2] の32 ビット浮動小数点数用テンソル）を作る。また、TF 関数の
get_concrete_function() メソッドを使えば、入力の特定の組合せのための具象関数が作
れる。この具象関数は通常の関数と同じように呼び出せるが、1 種類の入力シグネチャしかサポー
トしない（この場合は、32 ビット浮動小数点数スカラテンソル用）。

In [127]:
concrete_function = tf_cube.get_concrete_function(tf.constant(2.0))
concrete_function.graph

In [128]:
concrete_function(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [129]:
concrete_function is tf_cube.get_concrete_function(tf.constant(2.0))

True

図G-1 は、tf_cube(2) とtf_cube(tf.constant(2.0)) を呼び出したあとのtf_cube()TF
関数の状態を示している。それぞれのシグネチャのために全部で2 個の具象関数が生成され、それ
ぞれが最適化された関数グラフ（function graph：FuncGraph）と関数定義（function definition：
FunctionDef）を持っている。関数定義は、関数グラフのなかの入出力に対応する部分を指して
いる（破線の矢印）。個々のFuncGraph では、ノード（楕円で示されている）はオペレーション
（たとえば累乗、定数化、xのような引数のプレースホルダ）を表しエッジ（オペレーションとオペ
レーションを結ぶ実線の矢印）はグラフ内を動くテンソルを表している。左の具象関数はx=2に特
化されているので、TensorFlow は常に8 を出力するように単純化されている（関数定義に入力さ
え含まれていないことに注意しよう）。右の具象関数は、32 ビット浮動小数点数のスカラテンソル
に特化しており、これ以上単純化できない。tf_cube(tf.constant(5.0)) を呼び出すと、第2
の具象関数が呼び出され、xのためのプレースホルダオペレーションは5.0を出力し、累乗オペレー
ションが5.0**3を計算して、125.0 を出力する。

![G_1](G_1.png)

### Exploring Function Definitions and Graphs

In [214]:
concrete_function.graph

In [215]:
ops = concrete_function.graph.get_operations()
ops

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'pow/y' type=Const>,
 <tf.Operation 'pow' type=Pow>,
 <tf.Operation 'Identity' type=Identity>]

In [216]:
pow_op = ops[2]
list(pow_op.inputs)

[<tf.Tensor 'x:0' shape=() dtype=float32>,
 <tf.Tensor 'pow/y:0' shape=() dtype=float32>]

In [217]:
pow_op.outputs

[<tf.Tensor 'pow:0' shape=() dtype=float32>]

In [218]:
concrete_function.graph.get_operation_by_name('x')

<tf.Operation 'x' type=Placeholder>

In [219]:
concrete_function.graph.get_tensor_by_name('Identity:0')

<tf.Tensor 'Identity:0' shape=() dtype=float32>

In [220]:
concrete_function.function_def.signature

name: "__inference_cube_1073862"
input_arg {
  name: "x"
  type: DT_FLOAT
}
output_arg {
  name: "identity"
  type: DT_FLOAT
}

### How TF Functions Trace Python Functions to Extract Their Computation Graphs

In [221]:
@tf.function
def tf_cube(x):
    print("print:", x)
    return x ** 3

In [222]:
result = tf_cube(tf.constant(2.0))

print: Tensor("x:0", shape=(), dtype=float32)


In [223]:
result

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

In [224]:
result = tf_cube(2)
result = tf_cube(3)
result = tf_cube(tf.constant([[1., 2.]])) # New shape: trace!
result = tf_cube(tf.constant([[3., 4.], [5., 6.]])) # New shape: trace!
result = tf_cube(tf.constant([[7., 8.], [9., 10.], [11., 12.]])) # New shape: trace!


print: 2
print: 3
print: Tensor("x:0", shape=(1, 2), dtype=float32)
print: Tensor("x:0", shape=(2, 2), dtype=float32)


print: Tensor("x:0", shape=(3, 2), dtype=float32)


It is also possible to specify a particular input signature:

In [225]:
@tf.function(input_signature=[tf.TensorSpec([None, 28, 28], tf.float32)])
def shrink(images):
    print("Tracing", images)
    return images[:, ::2, ::2] # drop half the rows and columns

In [226]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [227]:
img_batch_1 = tf.random.uniform(shape=[100, 28, 28])
img_batch_2 = tf.random.uniform(shape=[50, 28, 28])
preprocessed_images = shrink(img_batch_1) # Traces the function.
preprocessed_images = shrink(img_batch_2) # Reuses the same concrete function.

Tracing Tensor("images:0", shape=(None, 28, 28), dtype=float32)


In [228]:
img_batch_3 = tf.random.uniform(shape=[2, 2, 2])
try:
    preprocessed_images = shrink(img_batch_3)  # rejects unexpected types or shapes
except ValueError as ex:
    print(ex)

Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[[0.7413678  0.62854624]
  [0.01738465 0.3431449 ]]

 [[0.51063764 0.3777541 ]
  [0.07321596 0.02137029]]], shape=(2, 2, 2), dtype=float32))
  input_signature: (
    TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None))


### Using Autograph To Capture Control Flow

A "static" `for` loop using `range()`:

In [229]:
@tf.function
def add_10(x):
    for i in range(10):
        x += 1
    return x

In [230]:
add_10(tf.constant(5))

<tf.Tensor: shape=(), dtype=int32, numpy=15>

In [231]:
add_10.get_concrete_function(tf.constant(5)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'add_1/y' type=Const>,
 <tf.Operation 'add_1' type=AddV2>,
 <tf.Operation 'add_2/y' type=Const>,
 <tf.Operation 'add_2' type=AddV2>,
 <tf.Operation 'add_3/y' type=Const>,
 <tf.Operation 'add_3' type=AddV2>,
 <tf.Operation 'add_4/y' type=Const>,
 <tf.Operation 'add_4' type=AddV2>,
 <tf.Operation 'add_5/y' type=Const>,
 <tf.Operation 'add_5' type=AddV2>,
 <tf.Operation 'add_6/y' type=Const>,
 <tf.Operation 'add_6' type=AddV2>,
 <tf.Operation 'add_7/y' type=Const>,
 <tf.Operation 'add_7' type=AddV2>,
 <tf.Operation 'add_8/y' type=Const>,
 <tf.Operation 'add_8' type=AddV2>,
 <tf.Operation 'add_9/y' type=Const>,
 <tf.Operation 'add_9' type=AddV2>,
 <tf.Operation 'Identity' type=Identity>]

A "dynamic" loop using `tf.while_loop()`:

In [232]:
@tf.function
def add_10(x):
    condition = lambda i, x: tf.less(i, 10)
    body = lambda i, x: (tf.add(i, 1), tf.add(x, 1))
    final_i, final_x = tf.while_loop(condition, body, [tf.constant(0), x])
    return final_x

In [233]:
add_10(tf.constant(5))

<tf.Tensor: shape=(), dtype=int32, numpy=15>

In [234]:
add_10.get_concrete_function(tf.constant(5)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'Const' type=Const>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

A "dynamic" `for` loop using `tf.range()` (captured by autograph):

In [235]:
@tf.function
def add_10(x):
    for i in tf.range(10):
        x = x + 1
    return x

In [236]:
add_10.get_concrete_function(tf.constant(0)).graph.get_operations()

[<tf.Operation 'x' type=Placeholder>,
 <tf.Operation 'range/start' type=Const>,
 <tf.Operation 'range/limit' type=Const>,
 <tf.Operation 'range/delta' type=Const>,
 <tf.Operation 'range' type=Range>,
 <tf.Operation 'sub' type=Sub>,
 <tf.Operation 'floordiv' type=FloorDiv>,
 <tf.Operation 'mod' type=FloorMod>,
 <tf.Operation 'zeros_like' type=Const>,
 <tf.Operation 'NotEqual' type=NotEqual>,
 <tf.Operation 'Cast' type=Cast>,
 <tf.Operation 'add' type=AddV2>,
 <tf.Operation 'zeros_like_1' type=Const>,
 <tf.Operation 'Maximum' type=Maximum>,
 <tf.Operation 'while/maximum_iterations' type=Const>,
 <tf.Operation 'while/loop_counter' type=Const>,
 <tf.Operation 'while' type=StatelessWhile>,
 <tf.Operation 'Identity' type=Identity>]

### Handling Variables and Other Resources in TF Functions

In [237]:
counter = tf.Variable(0)

@tf.function
def increment(counter, c=1):
    return counter.assign_add(c)

In [238]:
increment(counter)
increment(counter)

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [239]:
function_def = increment.get_concrete_function(counter).function_def
function_def.signature.input_arg[0]

name: "counter"
type: DT_RESOURCE

In [240]:
counter = tf.Variable(0)

@tf.function
def increment(c=1):
    return counter.assign_add(c)

In [241]:
increment()
increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [242]:
function_def = increment.get_concrete_function().function_def
function_def.signature.input_arg[0]

name: "assignaddvariableop_resource"
type: DT_RESOURCE

In [243]:
class Counter:
    def __init__(self):
        self.counter = tf.Variable(0)

    @tf.function
    def increment(self, c=1):
        return self.counter.assign_add(c)

In [244]:
c = Counter()
c.increment()
c.increment()

<tf.Tensor: shape=(), dtype=int32, numpy=2>

In [245]:
@tf.function
def add_10(x):
    for i in tf.range(10):
        x += 1
    return x

print(tf.autograph.to_code(add_10.python_function))

def tf__add(x):
    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = ag__.ld(x)
            x += 1
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [246]:
def display_tf_code(func):
    from IPython.display import display, Markdown
    if hasattr(func, "python_function"):
        func = func.python_function
    code = tf.autograph.to_code(func)
    display(Markdown('```python\n{}\n```'.format(code)))

In [247]:
display_tf_code(add_10)

```python
def tf__add(x):
    with ag__.FunctionScope('add_10', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body(itr):
            nonlocal x
            i = itr
            x = ag__.ld(x)
            x += 1
        i = ag__.Undefined('i')
        ag__.for_stmt(ag__.converted_call(ag__.ld(tf).range, (10,), None, fscope), None, loop_body, get_state, set_state, ('x',), {'iterate_names': 'i'})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)

```

## Using TF Functions with tf.keras (or Not)

By default, tf.keras will automatically convert your custom code into TF Functions, no need to use
`tf.function()`:

In [248]:
# Custom loss function
def my_mse(y_true, y_pred):
    print("Tracing loss my_mse()")
    return tf.reduce_mean(tf.square(y_pred - y_true))

In [249]:
# Custom metric function
def my_mae(y_true, y_pred):
    print("Tracing metric my_mae()")
    return tf.reduce_mean(tf.abs(y_pred - y_true))

In [250]:
# Custom layer
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.units),
                                      initializer='uniform',
                                      trainable=True)
        self.biases = self.add_weight(name='bias', 
                                      shape=(self.units,),
                                      initializer='zeros',
                                      trainable=True)
        super().build(input_shape)

    def call(self, X):
        print("Tracing MyDense.call()")
        return self.activation(X @ self.kernel + self.biases)

In [251]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [252]:
# Custom model
class MyModel(keras.models.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = MyDense(30, activation="relu")
        self.hidden2 = MyDense(30, activation="relu")
        self.output_ = MyDense(1)

    def call(self, input):
        print("Tracing MyModel.call()")
        hidden1 = self.hidden1(input)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input, hidden2])
        output = self.output_(concat)
        return output

model = MyModel()

In [253]:
model.compile(loss=my_mse, optimizer="nadam", metrics=[my_mae])

In [254]:
model.fit(X_train_scaled, y_train, epochs=2,
          validation_data=(X_valid_scaled, y_valid))
model.evaluate(X_test_scaled, y_test)

Epoch 1/2
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
340/363 [===========================>..] - ETA: 0s - loss: 2.8762 - my_mae: 1.2771Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
363/363 [==============================] - 1s 1ms/step - loss: 2.7755 - my_mae: 1.2455 - val_loss: 0.5569 - val_my_mae: 0.4819
Epoch 2/2
162/162 [==============================] - 0s 469us/step - loss: 0.4164 - my_mae: 0.4639


[0.4163525104522705, 0.4639028012752533]

You can turn this off by creating the model with `dynamic=True` (or calling `super().__init__(dynamic=True, **kwargs)` in the model's constructor):

In [255]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [256]:
model = MyModel(dynamic=True)

In [257]:
model.compile(loss=my_mse, optimizer="nadam", metrics=[my_mae])

Not the custom code will be called at each iteration. Let's fit, validate and evaluate with tiny datasets to avoid getting too much output:

In [258]:
model.fit(X_train_scaled[:64], y_train[:64], epochs=1,
          validation_data=(X_valid_scaled[:64], y_valid[:64]), verbose=0)
model.evaluate(X_test_scaled[:64], y_test[:64], verbose=0)

Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()


[5.507260322570801, 2.0566811561584473]

Alternatively, you can compile a model with `run_eagerly=True`:

In [259]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [260]:
model = MyModel()

In [261]:
model.compile(loss=my_mse, optimizer="nadam", metrics=[my_mae], run_eagerly=True)

In [262]:
model.fit(X_train_scaled[:64], y_train[:64], epochs=1,
          validation_data=(X_valid_scaled[:64], y_valid[:64]), verbose=0)
model.evaluate(X_test_scaled[:64], y_test[:64], verbose=0)

Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()
Tracing MyModel.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing MyDense.call()
Tracing loss my_mse()
Tracing metric my_mae()


[5.507260322570801, 2.0566811561584473]

## Custom Optimizers

Defining custom optimizers is not very common, but in case you are one of the happy few who gets to write one, here is an example:

In [263]:
class MyMomentumOptimizer(keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, momentum=0.9, name="MyMomentumOptimizer", **kwargs):
        """Call super().__init__() and use _set_hyper() to store hyperparameters"""
        super().__init__(name, **kwargs)
        self._set_hyper("learning_rate", kwargs.get("lr", learning_rate)) # handle lr=learning_rate
        self._set_hyper("decay", self._initial_decay) # 
        self._set_hyper("momentum", momentum)
    
    def _create_slots(self, var_list):
        """For each model variable, create the optimizer variable associated with it.
        TensorFlow calls these optimizer variables "slots".
        For momentum optimization, we need one momentum slot per model variable.
        """
        for var in var_list:
            self.add_slot(var, "momentum")

    @tf.function
    def _resource_apply_dense(self, grad, var):
        """Update the slots and perform one optimization step for one model variable
        """
        var_dtype = var.dtype.base_dtype
        lr_t = self._decayed_lr(var_dtype) # handle learning rate decay
        momentum_var = self.get_slot(var, "momentum")
        momentum_hyper = self._get_hyper("momentum", var_dtype)
        momentum_var.assign(momentum_var * momentum_hyper - (1. - momentum_hyper)* grad)
        var.assign_add(momentum_var * lr_t)

    def _resource_apply_sparse(self, grad, var):
        raise NotImplementedError

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "learning_rate": self._serialize_hyperparameter("learning_rate"),
            "decay": self._serialize_hyperparameter("decay"),
            "momentum": self._serialize_hyperparameter("momentum"),
        }

In [264]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [265]:
model = keras.models.Sequential([keras.layers.Dense(1, input_shape=[8])])
model.compile(loss="mse", optimizer=MyMomentumOptimizer())
model.fit(X_train_scaled, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 0s 444us/step - loss: 4.9648
Epoch 2/5
363/363 [==============================] - 0s 444us/step - loss: 1.7888
Epoch 3/5
363/363 [==============================] - 0s 437us/step - loss: 1.0021
Epoch 4/5
363/363 [==============================] - 0s 451us/step - loss: 0.7869
Epoch 5/5
363/363 [==============================] - 0s 446us/step - loss: 0.7122


# Exercises

## 1. to 11.
See Appendix A.

## 12. Implement a custom layer that performs _Layer Normalization_
_We will use this type of layer in Chapter 15 when using Recurrent Neural Networks._

### a.
_Exercise: The `build()` method should define two trainable weights *α* and *β*, both of shape `input_shape[-1:]` and data type `tf.float32`. *α* should be initialized with 1s, and *β* with 0s._

Solution: see below.

### b.
_Exercise: The `call()` method should compute the mean_ μ _and standard deviation_ σ _of each instance's features. For this, you can use `tf.nn.moments(inputs, axes=-1, keepdims=True)`, which returns the mean μ and the variance σ<sup>2</sup> of all instances (compute the square root of the variance to get the standard deviation). Then the function should compute and return *α*⊗(*X* - μ)/(σ + ε) + *β*, where ⊗ represents itemwise multiplication (`*`) and ε is a smoothing term (small constant to avoid division by zero, e.g., 0.001)._

In [266]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps

    def build(self, batch_input_shape):
        self.alpha = self.add_weight(
            name="alpha", shape=batch_input_shape[-1:],
            initializer="ones")
        self.beta = self.add_weight(
            name="beta", shape=batch_input_shape[-1:],
            initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        return self.alpha * (X - mean) / (tf.sqrt(variance + self.eps)) + self.beta

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}

Note that making _ε_ a hyperparameter (`eps`) was not compulsory. Also note that it's preferable to compute `tf.sqrt(variance + self.eps)` rather than `tf.sqrt(variance) + self.eps`. Indeed, the derivative of sqrt(z) is undefined when z=0, so training will bomb whenever the variance vector has at least one component equal to 0. Adding _ε_ within the square root guarantees that this will never happen.

### c.
_Exercise: Ensure that your custom layer produces the same (or very nearly the same) output as the `keras.layers.LayerNormalization` layer._

Let's create one instance of each class, apply them to some data (e.g., the training set), and ensure that the difference is negligeable.

In [267]:
X = X_train.astype(np.float32)

custom_layer_norm = LayerNormalization()
keras_layer_norm = keras.layers.LayerNormalization()

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=5.6045884e-08>

Yep, that's close enough. To be extra sure, let's make alpha and beta completely random and compare again:

In [268]:
random_alpha = np.random.rand(X.shape[-1])
random_beta = np.random.rand(X.shape[-1])

custom_layer_norm.set_weights([random_alpha, random_beta])
keras_layer_norm.set_weights([random_alpha, random_beta])

tf.reduce_mean(keras.losses.mean_absolute_error(
    keras_layer_norm(X), custom_layer_norm(X)))

<tf.Tensor: shape=(), dtype=float32, numpy=2.2921004e-08>

Still a negligeable difference! Our custom layer works fine.

## 13. Train a model using a custom training loop to tackle the Fashion MNIST dataset
_The Fashion MNIST dataset was introduced in Chapter 10._

### a.
_Exercise: Display the epoch, iteration, mean training loss, and mean accuracy over each epoch (updated at each iteration), as well as the validation loss and accuracy at the end of each epoch._

In [269]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test.astype(np.float32) / 255.

In [270]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [271]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

In [272]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

In [273]:
with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape() as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()


All epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/1718 [00:00<?, ?it/s]

### b.
_Exercise: Try using a different optimizer with a different learning rate for the upper layers and the lower layers._

In [274]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [275]:
lower_layers = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(100, activation="relu"),
])
upper_layers = keras.models.Sequential([
    keras.layers.Dense(10, activation="softmax"),
])
model = keras.models.Sequential([
    lower_layers, upper_layers
])

In [276]:
lower_optimizer = keras.optimizers.SGD(learning_rate=1e-4)
upper_optimizer = keras.optimizers.Nadam(learning_rate=1e-3)

In [277]:
n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

In [278]:
with trange(1, n_epochs + 1, desc="All epochs") as epochs:
    for epoch in epochs:
        with trange(1, n_steps + 1, desc="Epoch {}/{}".format(epoch, n_epochs)) as steps:
            for step in steps:
                X_batch, y_batch = random_batch(X_train, y_train)
                with tf.GradientTape(persistent=True) as tape:
                    y_pred = model(X_batch)
                    main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
                    loss = tf.add_n([main_loss] + model.losses)
                for layers, optimizer in ((lower_layers, lower_optimizer),
                                          (upper_layers, upper_optimizer)):
                    gradients = tape.gradient(loss, layers.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, layers.trainable_variables))
                del tape
                for variable in model.variables:
                    if variable.constraint is not None:
                        variable.assign(variable.constraint(variable))                    
                status = OrderedDict()
                mean_loss(loss)
                status["loss"] = mean_loss.result().numpy()
                for metric in metrics:
                    metric(y_batch, y_pred)
                    status[metric.name] = metric.result().numpy()
                steps.set_postfix(status)
            y_pred = model(X_valid)
            status["val_loss"] = np.mean(loss_fn(y_valid, y_pred))
            status["val_accuracy"] = np.mean(keras.metrics.sparse_categorical_accuracy(
                tf.constant(y_valid, dtype=np.float32), y_pred))
            steps.set_postfix(status)
        for metric in [mean_loss] + metrics:
            metric.reset_states()

All epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 2/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 3/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 4/5:   0%|          | 0/1718 [00:00<?, ?it/s]

Epoch 5/5:   0%|          | 0/1718 [00:00<?, ?it/s]